# Make RGB NIRCam stamps showing slit footprint
We will be using the `trilogy` package to make RGB stamps and the shutter footprint regions.
The information we need to show the footprint and the source (optional) is all in the "shutters" files.


In [ ]:
from trilogy import trilogy
from astropy.io import fits
from astropy.wcs import WCS
from astropy.table import Table
from astropy import table
from astropy.visualization import make_lupton_rgb

from astropy.coordinates import SkyCoord
import astropy.units as u
import glob

from collections import defaultdict
import matplotlib
from matplotlib import pyplot as plt
import PIL  # Python Image Library
from PIL import Image
import os, sys
import contextlib

from astropy.nddata import Cutout2D
import numpy as np
from scipy.ndimage import zoom
import pandas as pd
import scipy
import os
import glob
import json
import numpy as np


## Functions to make cutouts

In [ ]:
# helper functions
def roundint(x):
    return int(np.round(x))

def slices_extent(x, y, dx, dy=0):
    dy = dy or dx
    xlo = roundint(x-dx)
    xhi = roundint(x+dx+1)
    ylo = roundint(y-dy)
    yhi = roundint(y+dy+1)
    xslice = slice(xlo, xhi)
    yslice = slice(ylo, yhi)
    slices = yslice, xslice
    extent = xlo, xhi, ylo, yhi
    return slices, extent

In [ ]:
def make_stamp(imagesRGB, outfile='stamp.pdf'):
    """
    Make RGB stamp using trilogy
    """
    
    ##################
    # MAKE RGB IMAGE #
    ##################
    
    # set luminosity of the noise in each channel
    noiselums = {'R': 0.2, 'G': 0.2, 'B': 0.2}
        
    #with open(os.devnull, "w") as fnull, contextlib.redirect_stdout(fnull): # avoid printing out stuff
    trilogy.Trilogy(
        infile = None, samplesize = 20000, stampsize = 20000, maxstampsize = 20000, \
        deletetests = 1, deletefilters = 1, testfirst = 0, showwith = "PIL", \
        mode = 'RGB', imagesorder = 'RGB', imagesRGB = imagesRGB, noiselums = noiselums, images = None, \
        outname = 'color_image_temp', satpercent = 0.000, noiselum = 0.5, noisesig = 10, \
        noisesig0 = 10, correctbias = 0, colorsatfac = 1, combine = 'sum', show = False
        ).run()
    
    # read in RGB image and delete png file
    im = Image.open('color_image_temp.png')
    im = im.transpose(method=Image.FLIP_TOP_BOTTOM)
    NIRCam_image = np.asarray(im)
    os.remove('color_image_temp.png')
    
    # read WCS from a single FITS file
    single_filename = imagesRGB['R'][0].split('[')[0] # remove '[1]' if present
    image_hdulist = fits.open(single_filename)
    image_wcs = WCS(image_hdulist[1].header, image_hdulist)
    
    # --- Plot using matplotlib and save ---
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.imshow(NIRCam_image, origin='lower')
    ax.axis('off')  # Hide axes
    
    # Save the RGB image as PDF
    fig.savefig(outfile, bbox_inches='tight', pad_inches=0.0)
    plt.close(fig)  # Clean up the figure




## Start making the stamps

In [ ]:

# ---------- SETUP ----------
input_dir = "/home/bpc/University/master/Red_Cardinal/cutouts_3x3/"
output_dir = "/home/bpc/University/master/Red_Cardinal/stamps/"
os.makedirs(output_dir, exist_ok=True)

# ---------- STEP 1: Get IDs from NIRCam, PRIMER and COSMOS-Web ----------
f444w_files = glob.glob(os.path.join(input_dir, "*F444W*.fits"))
f444w_ids = [os.path.basename(f).split("_F444W")[0] for f in f444w_files]

f770w_primer_files = glob.glob(os.path.join(input_dir, "*F770W_cutout_primer.fits"))
f770w_primer_ids = [os.path.basename(f).split("_F770W")[0] for f in f770w_primer_files]

f770w_cweb_files = glob.glob(os.path.join(input_dir, "*F770W_cutout_cweb.fits"))
f770w_cweb_ids = [os.path.basename(f).split("_F770W")[0] for f in f770w_cweb_files]

f1800w_files = glob.glob(os.path.join(input_dir, "*F1800W*.fits"))
f1800w_ids = [os.path.basename(f).split("_F1800W")[0] for f in f1800w_files]

# Remove all galaxies in f770w_primer_ids from f770w_cweb_ids
f770w_cweb_ids = np.setdiff1d(f770w_cweb_ids, f770w_primer_ids)


# ---------- STEP 2: Collect MIRI and NIRCam Files ----------
fits_files = glob.glob(os.path.join(input_dir, "*.fits"))
filters = ["F444W", "F770W", "F1800W"]


Check extensions of FITS files

In [ ]:
r_file = "/home/bpc/University/master/Red_Cardinal/cutouts_3x3/18769_F1800W_cutout_primer.fits"
with fits.open(r_file) as hdul:
    for i, hdu in enumerate(hdul):
        print(f"HDU {i}: {hdu.name}, shape = {getattr(hdu.data, 'shape', None)}")

g_file = "/home/bpc/University/master/Red_Cardinal/cutouts_3x3/18769_F770W_cutout_primer.fits"
with fits.open(g_file) as hdul:
    for i, hdu in enumerate(hdul):
        print(f"HDU {i}: {hdu.name}, shape = {getattr(hdu.data, 'shape', None)}")
        
b_file = "/home/bpc/University/master/Red_Cardinal/cutouts_3x3/18769_F444W_cutout_nircam.fits"
with fits.open(b_file) as hdul:
    for i, hdu in enumerate(hdul):
        print(f"HDU {i}: {hdu.name}, shape = {getattr(hdu.data, 'shape', None)}")

Function block:

1. Upscale the pixels from (27,27) to (100,100)
2. Load FITS data
3. Store updated data in a new FITS file

In [ ]:
# Function to upscale an image to match the target pixel scale
def upscale_image_to_pixel_scale(image, original_pixel_scale, target_pixel_scale):
   # Resample factor
    resample_factor = float(target_pixel_scale) / original_pixel_scale
    print(resample_factor)
    # Rescale the image
    rescaled_image = zoom(image, resample_factor, order=1)  # order=3 for cubic interpolation
    
    # Clamp values to the valid range (e.g., 0 to 65535 for 16-bit)
    rescaled_image = np.clip(rescaled_image, 0, 65535)
    
    return rescaled_image


# Function to bin an image into smaller pixels
def bin_image(image, bin_factor):
    """
    Bins the image by averaging over 'bin_factor' pixels in each direction.
    """
    shape = (image.shape[0] // bin_factor, image.shape[1] // bin_factor)
    binned_image = image[:shape[0] * bin_factor, :shape[1] * bin_factor].reshape(shape[0], bin_factor, shape[1], bin_factor)
    return binned_image.mean(axis=(1, 3))  # Take mean of each bin


# Function to subdivide each pixel into smaller subpixels
def subdivide_pixels(image, subpixel_factor):
    """
    Subdivides each pixel into smaller subpixels (e.g., 2x2, 4x4), and redistributes the brightness.
    """
    new_shape = (image.shape[0] * subpixel_factor, image.shape[1] * subpixel_factor)
    subpixel_image = np.zeros(new_shape)

    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            subpixel_image[i * subpixel_factor:(i + 1) * subpixel_factor, j * subpixel_factor:(j + 1) * subpixel_factor] = image[i, j]
    
    return subpixel_image

# Load image data from FITS files
def load_fits_data(file_path):
    with fits.open(file_path) as hdul:
        return hdul[1].data, hdul[1].header


# Save the updated image data to a new FITS file
def save_fits_data(output_file, image_data, header):
    image_data = image_data.astype(np.uint16)
    hdu = fits.PrimaryHDU(image_data, header=header)
    hdul = fits.HDUList([hdu])
    hdul.writeto(output_file, overwrite=True)
    
    
def resample_and_save_fits(input_fits_path, output_fits_path, output_shape):
    """
    Resample a FITS image to a new shape and update the WCS in the header accordingly.
    
    Parameters:
    -----------
    input_fits_path : str
        Path to the original FITS file.
    output_fits_path : str
        Path to save the resampled FITS file.
    output_shape : tuple
        Desired output shape as (ny, nx), e.g. (100, 100).
    """
    with fits.open(input_fits_path) as hdul:
        data = hdul[1].data
        header = hdul[1].header.copy()

    # Original shape
    ny_old, nx_old = data.shape
    ny_new, nx_new = output_shape

    # Compute zoom factors
    zoom_y = ny_new / ny_old
    zoom_x = nx_new / nx_old

    # Resample data
    resampled_data = zoom(data, (zoom_y, zoom_x), order=1, mode='nearest')
    # Update header
    if 'CDELT1' in header and 'CDELT2' in header:
        header['CDELT1'] /= zoom_x
        header['CDELT2'] /= zoom_y
    if 'CRPIX1' in header and 'CRPIX2' in header:
        header['CRPIX1'] *= zoom_x
        header['CRPIX2'] *= zoom_y

    # Save to new FITS file
    hdu = fits.PrimaryHDU(data=resampled_data, header=header)
    hdu.writeto(output_fits_path, overwrite=True)

    print(f"Saved resampled FITS to {output_fits_path} ({ny_old}x{nx_old} → {ny_new}x{nx_new})")


In [ ]:
r_file = os.path.join(input_dir, "9871_F1800W_cutout_primer.fits")

# Load FITS files into NumPy arrays
r_data, r_header = load_fits_data(r_file)

# Number of pixels per image
nircam_pixel_scale = 100  # NIRCam pixel scale in arcseconds/pixel
miri_pixel_scale = 27     # MIRI pixel scale in arcseconds/pixel

# Upscale the images to the target pixel scale
#r_rescaled = upscale_image_to_pixel_scale(r_file, miri_pixel_scale, nircam_pixel_scale)
#r_rescaled = bin_image(r_data, 2)
#r_rescaled = subdivide_pixels(r_data, 4)


# Save the rescaled images back to new FITS files
output_r_file = os.path.join(output_dir, f"9871_F1800W_cutout_primer_rescaled.fits")
#save_fits_data(output_r_file, r_rescaled, r_header)

resample_and_save_fits(r_file, output_r_file, (100,100))


with fits.open(r_file) as hdul:
    data = hdul[1].data
    print(data)

with fits.open(output_r_file) as hdul:
    data = hdul[0].data
    print(data)

Function which calls the make_stamps function iteratively

In [ ]:

gal_id = "18769"
try:
    print(f"Creating image for {gal_id}...")
    
    # Construct base filenames (without [0])
    r_file_base = os.path.join(input_dir, f"{gal_id}_F1800W_cutout_primer.fits")
    g_file_primer_base = os.path.join(input_dir, f"{gal_id}_F770W_cutout_primer.fits")
    g_file_cweb_base = os.path.join(input_dir, f"{gal_id}_F770W_cutout_cweb.fits")
    b_file_base = os.path.join(input_dir, f"{gal_id}_F444W_cutout_nircam.fits")

    # Decide which F770W file to use
    if gal_id in f770w_primer_ids and os.path.exists(g_file_primer_base):
        g_file = g_file_primer_base + "[1]"
    elif gal_id in f770w_cweb_ids and os.path.exists(g_file_cweb_base):
        g_file = g_file_cweb_base + "[1]"
    else:
        print(f"[Skipping {gal_id}] No valid F770W file found.")
        #continue

    # Now safely add [0] to files
    r_file = r_file_base + "[1]"
    b_file = b_file_base + "[0]"

    # Upscale the pixels to be all in the same format
    f_file = upscale_image_to_pixel_scale(r_file, miri_pixel_scale, nircam_pixel_scale)
    g_file = upscale_image_to_pixel_scale(g_file, miri_pixel_scale, nircam_pixel_scale)
    
    # Stack into imagesRGB dictionary
    imagesRGB = {'R': [r_file], 
                    'G': [g_file], 
                    'B': [b_file]}
    
    # Call your image function (without shutters)
    outfile = os.path.join(output_dir, f"{gal_id}_stamp.pdf")
    make_stamp(imagesRGB, outfile)

except Exception as e:
    print(f"Error processing {gal_id}: {e}")


Actually produce the RGB images:

In [ ]:
# Number of pixels per image
nircam_pixel_scale = 100  # NIRCam pixel scale in arcseconds/pixel
miri_pixel_scale = 27     # MIRI pixel scale in arcseconds/pixel


# Function to upscale an image to match the target pixel scale
def upscale_image_to_pixel_scale(image, original_pixel_scale, target_pixel_scale):
    resample_factor = float(target_pixel_scale) / original_pixel_scale
    return zoom(image, resample_factor, order=3)  # order=3 for cubic interpolation

# Step 3: Loop over galaxies
for gal_id, filters in galaxies.items():

    # Try to assign R, G, B from available filters
    rgb_data = {}
    used_filters = {}

    for colour, choices in filter_map.items():
        for filt in choices:
            if filt in filters:
                with fits.open(filters[filt]) as hdul:
                    if filt == "F444W": index = 0
                    else: index = 1
                    data = hdul[index].data
                rgb_data[colour] = data
                used_filters[colour] = filt
                print(f"Assigned {filt} to {colour} channel")  # Debugging line
                break

    # Debugging output to check rgb_data
    #print(f"rgb_data for {gal_id}: {rgb_data}")
    print(f"used_filters for {gal_id}: {used_filters}")

    # Upscale MIRI images to match NIRCam pixel scale
    if set(['R', 'G', 'B']).issubset(rgb_data):
        try:
            # Upscale MIRI data to match NIRCam resolution
            if 'R' in rgb_data and used_filters['R'] in ["F770W", "F1800W"]:
                rgb_data['R'] = upscale_image_to_pixel_scale(
                    rgb_data['R'], miri_pixel_scale, nircam_pixel_scale
                )
            if 'G' in rgb_data and used_filters['G'] in ["F770W", "F1800W"]:
                rgb_data['G'] = upscale_image_to_pixel_scale(
                    rgb_data['G'], miri_pixel_scale, nircam_pixel_scale
                )
            if 'B' in rgb_data and used_filters['B'] in ["F770W", "F1800W"]:
                rgb_data['B'] = upscale_image_to_pixel_scale(
                    rgb_data['B'], nircam_pixel_scale
                )

            # Generate the RGB image
            rgb_image = make_lupton_rgb(
                rgb_data['R'], rgb_data['G'], rgb_data['B'],
                stretch=5, Q=10
            )

            # Plot and save the image
            plt.figure(figsize=(5, 5))
            plt.imshow(rgb_image, origin='lower')
            legend = f"R: {used_filters['R']}, G: {used_filters['G']}, B: {used_filters['B']}"
            plt.text(0.02, 0.95, legend, transform=plt.gca().transAxes,
                     fontsize=8, color='white', backgroundcolor='black', ha='left', va='top')
            plt.axis('off')
            plt.tight_layout()
            plt.savefig(os.path.join(output_dir, f"{gal_id}.pdf"), bbox_inches='tight', pad_inches=0.0)
            plt.close()

        except ValueError as e:
            print(f"Error creating RGB for {gal_id}: {e}")
    else:
        print(f"Skipping {gal_id}: incomplete RGB filters.")


In [ ]:
nircam_fits = "/home/bpc/University/master/Red_Cardinal/cutouts_3x3/8338_F444W_cutout_3x3.fits"
miri_770_fits = "/home/bpc/University/master/Red_Cardinal/cutouts_3x3/8338_F770W_cutout_primer004.fits"
miri_1800_fits = "/home/bpc/University/master/Red_Cardinal/cutouts_3x3/8338_F1800W_cutout_primer004.fits"

with fits.open(nircam_fits) as nircam_hdul:
    nircam_data = nircam_hdul[0].data
    print(nircam_data)

with fits.open(miri_770_fits) as miri_770_hdul:
    miri_770_data = miri_770_hdul[1].data
    print(miri_770_data)

with fits.open(miri_1800_fits) as miri_1800_hdul:
    miri_1800_data = miri_1800_hdul[1].data
    print(miri_1800_data)

In [ ]:
# read in shutter footprints
import regions, pickle
with open('shutters_bluejay/shutters_regions.pkl', 'rb') as f:
    shutters = pickle.load(f)
print(len(shutters))


In [ ]:
# loop through the list of shutters
for s in shutters:

    # read shutters into regions objects
    reg_source = regions.Regions.parse(s['source'], format='ds9')
    reg_source = reg_source[0] # list of regions with only one entry
    reg_aperture = regions.Regions.parse(s['AB_aperture'], format='ds9')
    
    # read FITS files with NIRCam cutouts
    imagesRGB = {"R": [f"v1.0_JWST_cutouts/{s['ID']}_F444W_cutout.fits[1]"], \
                 "G": [f"v1.0_JWST_cutouts/{s['ID']}_F200W_cutout.fits[1]"], \
                 "B": [f"v1.0_JWST_cutouts/{s['ID']}_F115W_cutout.fits[1]"]}
    
    # make footprint
    make_stamp_with_shutters(imagesRGB, reg_source, reg_aperture,
                             outfile=f"stamps/{s['ID']}.pdf", sourcemark=False, size_arcsec=3.0)

